In [1]:
import csv as csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
import scipy.optimize as opt  
from sklearn import metrics, linear_model, tree, ensemble
from sklearn.grid_search import GridSearchCV

/opt/conda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/opt/conda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
def get_ticket_aggr(initTrain, initFinal):
    trainFrame = pd.concat([initTrain.drop(['Survived'], axis = 1), initFinal])
    a1 = trainFrame.groupby('Ticket')['PassengerId'].count()
    a2 = initTrain.groupby('Ticket')['Survived'].sum()
    agg = a1.to_frame().join(a2.to_frame()).reset_index()
    agg = agg.rename(columns={'PassengerId': 'TicketCount', 'Survived': 'TicketSurvived'})
    agg.loc[agg.TicketSurvived.isnull(), 'TicketSurvived'] = 1
    agg['TicketSurvived?'] = (agg.TicketSurvived > 1).astype(bool)
    agg['TicketCount'] = agg['TicketCount'].astype(int)
    agg = agg.drop(['TicketSurvived', 'TicketSurvived?'], axis = 1)
    return agg

def get_surname_aggr(initTrain, initFinal):
    trainFrame = pd.concat([initTrain.drop(['Survived'], axis = 1), initFinal])
    a3 = trainFrame.groupby('surname')['PassengerId'].count()
    a4 = initTrain.groupby('surname')['Survived'].sum()
    agg2 = a3.to_frame().join(a4.to_frame()).reset_index()
    agg2 = agg2.rename(columns={'PassengerId': 'SurCount', 'Survived': 'SurSurvived'})
    agg2.loc[agg2.SurSurvived.isnull(), 'SurSurvived'] = 1
    agg2['SurSurvived?'] = (agg2.SurSurvived > 1).astype(bool)
    agg2['SurCount'] = agg2['SurCount'].astype(int)
    agg2 = agg2.drop(['SurSurvived', 'SurSurvived?'], axis = 1)    
    return agg2

def transform_0(dFrame):
    t1 = dFrame.Name.str.split(',', expand=True)
    t1.columns = ['surname', 'GivenName']
    t1['title'] = t1.GivenName.str.extract('(Mrs|Mr|Miss|Master)', expand=False)
    dFrame = pd.concat([t1, dFrame], axis=1)
    return dFrame

def transform_1(dFrame, catTypes, ticket_agg, surname_agg):
# Extracting features from Name
    dFrame['FamilySize'] = dFrame['SibSp'] + dFrame['Parch']
    dFrame = dFrame.drop(['SibSp', 'Parch'], axis=1) 
    dFrame = dFrame.drop(['PassengerId', 'Name','GivenName', 'Cabin'], axis=1) 
    dFrame['Child'] = dFrame.Age<16
    dFrame['Child'] = dFrame['Child'].astype(bool)
    # Factoring in survivors on the same ticket
    dFrame = pd.merge(dFrame, ticket_agg, how='left', on='Ticket')


    # Factoring in survivors on the same surname
    dFrame = pd.merge(dFrame, surname_agg, how='left', on='surname')
    
    # Fill NA for string fields
    for cat in catTypes:
        dFrame.loc[dFrame[cat].isnull(), cat] = 'NA'

    # Fill fare
    dFrame.loc[dFrame.Fare.isnull(),'Fare'] = dFrame.Fare.median()
    # Filling Age where it is null
    dFrame['AgeIsNull'] = pd.isnull(dFrame.Age).astype(int)

    dFrame['EmbarkedIsNull'] = pd.isnull(dFrame.Embarked).astype(int)    
    
    dFrame['Gender'] = dFrame['Sex'].map({'male': 1, 'female': 0}).astype(int)
    median_ages = np.zeros((2,3))
    for i in range(0, 2):
        for j in range(0, 3):
            median_ages[i,j] = dFrame[(dFrame['Gender'] == i) & \
                              (dFrame['Pclass'] == j+1)]['Age'].dropna().median()
    for i in range(0, 2):
        for j in range(0, 3):
            dFrame.loc[ (dFrame.Age.isnull()) & (dFrame.Gender == i) & (dFrame.Pclass == j+1),\
                'Age'] = median_ages[i,j]
    dFrame = dFrame.drop(['Ticket', 'surname', 'Gender'], axis=1)
    dFrame['Age*Class'] = dFrame.Age * dFrame.Pclass
    return dFrame
    
def getColumns(t1, t2):
    #INIT CATEGORY DICTIONARY
    dict = {}
    for cat in catTypes:
        cct = list(t1[cat].unique()) + list(t2[cat].unique())
        dict[cat] = set(cct)
    return dict

def trainAndTest(clf, test_size=20):
    x_train = Xtrain[:test_size]
    y_train = ytrain[:test_size]
    x_test = Xtrain[test_size:]
    y_test = ytrain[test_size:]
    clf.fit(x_train, y_train)
    print (clf.best_score_, clf.best_params_)
    model = clf.best_estimator_
    predictions = model.predict(x_train)  
    name = type(model).__name__
    print(name)
    print("Train Accuracy {x}, F1 score {f}".format(x=metrics.accuracy_score(predictions, y_train), f=metrics.f1_score(predictions, y_train)))
    predictions_test = model.predict(x_test)  

    accuracy = metrics.accuracy_score(predictions_test, y_test)
    f1_score = metrics.f1_score(predictions_test, y_test)
    print("Test Accuracy {x}, F1 score {f}".format(x=accuracy, f=f1_score))
    return {'accuracy': accuracy, f1_score: f1_score, 'model': model, 'name': name, 'predictions': predictions_test}

### Train Data set and calculate accuracy

In [3]:
# Constants
random_max_depth = 10
test_size = 267

catTypes = ['Pclass', 'Sex', 'title', 'Embarked']

hotTypes = ['Pclass', 'Sex']

initTrain = pd.read_csv('./data/train.csv', header=0)
print("-1) train shape {}".format(initTrain.shape))
initFinal = pd.read_csv('./data/test.csv', header=0)
print("-1) final set shape {}".format(initFinal.shape))

transTrain = transform_0(initTrain)
print("0) train transformed {}".format(transTrain.shape))
transFinal = transform_0(initFinal)
print("0) final transformed {}".format(transFinal.shape))


ticket_agg = get_ticket_aggr(transTrain, transFinal)

final_ticket_agg = get_ticket_aggr(transTrain, transFinal)

sur_agg = get_surname_aggr(transTrain, transFinal)

final_sur_agg = get_surname_aggr(transTrain, transFinal)

transTrain = transform_1(transTrain,  catTypes, ticket_agg, sur_agg)
print("1) train transformed {}".format(transTrain.shape))
transFinal = transform_1(transFinal, catTypes, final_ticket_agg, final_sur_agg)
print("1) final transformed {}".format(transFinal.shape))

catDict = getColumns(transTrain, transFinal)

columns = transTrain.drop(hotTypes+['Survived'], axis=1).columns

# Label encoding for string types
for cat in catTypes:
    lenc = LabelEncoder()
    lenc.fit(list(catDict[cat]))
    transTrain[cat] = lenc.transform(transTrain[cat])
    transFinal[cat] = lenc.transform(transFinal[cat])

    
Xtrain = transTrain.drop(hotTypes+['Survived'], axis=1).values
ytrain = transTrain.Survived.values
Xfinal = transFinal.drop(hotTypes, axis=1).values

# Hot encoding for few features
for cat in hotTypes:
    enc = OneHotEncoder(dtype=np.bool)
    enc.fit(transTrain[cat].reshape(-1,1))
    for r in range(0,enc.n_values_):
        columns = np.append(columns, "{0}_{1}".format(cat, r))  
    train_cat_features = enc.transform(transTrain[cat].reshape(-1,1)).toarray()
    test_cat_features = enc.transform(transFinal[cat].reshape(-1,1)).toarray()    
    Xtrain = np.concatenate([Xtrain,train_cat_features], axis=1)    
    Xfinal = np.concatenate([Xfinal,test_cat_features], axis=1)
    


print("2) training transformed {}".format(Xtrain.shape))
print("2)final set transformed{}".format(Xfinal.shape))


-1) train shape (891, 12)
-1) final set shape (418, 11)
0) train transformed (891, 15)
0) final transformed (418, 14)
1) train transformed (891, 14)
1) final transformed (418, 13)
2) training transformed (891, 16)
2)final set transformed(418, 16)


/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:53: DeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future


In [4]:
linear_param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }
decision_tree_param_grid = {'max_depth':list(range(2,20))}
random_param_grid = { 
    'n_estimators': [5, 10, 50, 100],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth':list(range(2,10))
}
classifiers = [
    GridSearchCV(linear_model.LogisticRegression(solver='liblinear', penalty = 'l2'), linear_param_grid),
    GridSearchCV(tree.DecisionTreeClassifier(), decision_tree_param_grid),
    GridSearchCV(ensemble.RandomForestClassifier(), random_param_grid)
]
accuracy = 0
model = None
results = np.array([])
for classifier in classifiers: 
    result = trainAndTest(classifier, test_size)
    results = np.append(results, result)
    if(result['accuracy'] > accuracy):
        accuracy = result['accuracy']
        model = result['model']

0.786516853933 {'C': 1}
LogisticRegression
Train Accuracy 0.8164794007490637, F1 score 0.7200000000000001
Test Accuracy 0.8012820512820513, F1 score 0.7244444444444443
0.797752808989 {'max_depth': 3}
DecisionTreeClassifier
Train Accuracy 0.8239700374531835, F1 score 0.6928104575163399
Test Accuracy 0.8173076923076923, F1 score 0.7298578199052134
0.820224719101 {'max_depth': 4, 'max_features': 'sqrt', 'n_estimators': 50}
RandomForestClassifier
Train Accuracy 0.846441947565543, F1 score 0.770949720670391
Test Accuracy 0.8349358974358975, F1 score 0.778494623655914


In [5]:
model = ensemble.RandomForestClassifier(max_depth=3, max_features='log2', n_estimators=5)
model.fit(Xtrain, ytrain)
yFinal = model.predict(Xfinal) 
output =  pd.DataFrame({'PassengerId': initFinal.PassengerId, 'Survived':yFinal})
output.groupby('Survived').size()

Survived
0    292
1    126
dtype: int64

In [6]:
model = linear_model.LogisticRegression(C=1,solver='liblinear', penalty = 'l2')
model.fit(Xtrain, ytrain)
yFinal = model.predict(Xfinal) 
output =  pd.DataFrame({'PassengerId': initFinal.PassengerId, 'Survived':yFinal})
output.groupby('Survived').size()

Survived
0    263
1    155
dtype: int64

In [7]:
model = tree.DecisionTreeClassifier(max_depth=3)
model.fit(Xtrain, ytrain)
yFinal = model.predict(Xfinal) 
output =  pd.DataFrame({'PassengerId': initFinal.PassengerId, 'Survived':yFinal})
output.groupby('Survived').size()

# Decision tree visualisation
# tree.export_graphviz(model, './data/tree.dot', feature_names=columns)
# dot -Tpng data/tree.dot -o data/tree.png


Survived
0    247
1    171
dtype: int64

In [8]:
#output.to_csv('./data/titanic_{0}_{1}.csv'.format(type(model).__name__, accuracy), index=False)